In [28]:
import os
import requests
from bs4 import BeautifulSoup
import json 
from dotenv import load_dotenv
from typing import List
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [29]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key wrking fine")
else:
    print("Check api key again")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key wrking fine


In [30]:
header={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
}

class Website:
    def __init__(self,url):
        self.url=url
        response=requests.get(url, headers=header)
        self.body=response.content
        soup=BeautifulSoup(self.body, 'html.parser')
        self.title=soup.title.string if soup.title else "No title found"
        if soup.body:
            for faaltu in soup.body(['script', 'style','img','input']):
                faaltu.decompose()
            self.text=soup.body.get_text(separator='\n', strip=True)
        else:
            self.text=""
        links=[link.get('href') for link in soup.find_all('a')]
        self.links=[link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [31]:
system_prompt="You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"

system_prompt+='You should respond in JSON as in this example:'
system_prompt+="""
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [32]:
def get_user_prompt(website):
    user_prompt=f'Here is the list of links on the website of {website.url} - '
    user_prompt+="please decide which of these are relevant web links  for a brochure about the company, respond with the full https URL in JSON format. \
        do not include Terms of Service, Privacy, email links.\n"
    user_prompt+="Links (some might be relative links):\nGive the output strictly in python list form"
    user_prompt+="\n".join(website.links)

    return user_prompt

In [33]:
def get_links(url):
    website=Website(url)
    response=openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content":system_prompt},
            {"role":"user", "content": get_user_prompt(website)}
        ],

            response_format={"type":"json_object"}
    )
    result=response.choices[0].message.content
    return json.loads(result)
    

In [34]:
class ConnectTimeout(Exception):
    """Custom exception for connection timeout."""
    pass


In [35]:
import time

def retry(url, retries=5, delay=2):
    for attempt in range(retries):
        try:
            return Website(url).get_contents()  
        except (ConnectTimeout, TimeoutError, Exception) as e:
            print(f"Error occurred for {url}: {e}. Attempt {attempt + 1} of {retries}.")
            time.sleep(delay)  
    print(f"Skipping {url} after {retries} failed attempts.")
    return f"Failed to retrieve contents for {url}\n"

def get_details(url):
    result = "Landing page:\n"
    result += retry(url)  
    links = get_links(url)
    print("Found links:", links)
    
    key_to_check = next(iter(links), None) 
    
    if key_to_check == 'links':
        for link in links["links"]:
            result += f"\n\n{link}\n"
            result += retry(link)  
    elif key_to_check == 'relevant_links':
        for link in links["relevant_links"]:
            result += f"\n\n{link}\n"
            result += retry(link)  
    else:
        result += "\nNo relevant links found.\n"
    
    return result


In [36]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a website ( it can be company website, college club website, someone's portfolio) so be prepared \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [37]:
def brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a professional brief brochure of the website in markdown.At footer always give the contacts link of their social media handles (except phone number) if available\n"
    user_prompt += get_details(url)
    user_prompt = user_prompt[:5_000] 
    return user_prompt

In [38]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [39]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [40]:
stream_brochure("RoboVitics", "https://robovitics.in/")

Found links: {'result': ['https://robovitics.in/#home', 'https://robovitics.in/#about-page', 'https://robovitics.in/#events-page', 'https://robovitics.in/#projects-page', 'https://robovitics.in/#orcus-page', 'https://robovitics.in/#kartavya-page', 'https://robovitics.in/#sponsors-page', 'https://robovitics.in/#memories', 'https://twitter.com/robo_vit', 'https://www.facebook.com/robovitics', 'https://www.instagram.com/robovitics/', 'https://www.linkedin.com/company/robovitics/', 'https://www.youtube.com/channel/UCFiwOI-W5b06NweratR-RdA', 'https://github.com/RoboVITics']}


# RoboVitics: Where Robots go to Party and Innovate

**Welcome to RoboVitics!** 

Are you ready to embark on the ultimate adventure of learning, growing, and battling it out with robots? You better be!

---

## About Us

At RoboVitics, we live by two powerful mottos:  
**“A place to learn, a chance to grow”** and **“Innovation is when Imagination meets Ambition.”** 

Founded in the hallowed halls of Vellore Institute of Technology (VIT) in 1984, our mission (should you choose to accept it) is to sprinkle a little tech magic and significantly improve students’ lives through exhilarating education and groundbreaking research. We bring together nerds, dreamers, and future tech magnates who want to unleash their inner creativity!

---

### Our Culture

Picture this: Our club is a lively tech hive buzzing with energy, where you can learn while having a jolly good time! We host workshops, seminars, and all sorts of techie shenanigans to make every student feel right at home. But don’t just take our word for it, our previous members have left behind a legacy that would make even the Terminator proud!

---

## Events

Grab your helmets and get ready for a whirlwind of fun at our electrifying events!

#### **Equinox**
36 hours of tech love, mentorship, and sharing ideas that’ll make your brain do cartwheels! 

#### **Vortex360**
Where designers can get their creative juices flowing and make CAD magic happen! Who says design can’t dazzle?

#### **Hands On Robotics**
If you ever wanted to build a robot that obeys your every command (in a safe and lawful way, of course), this is your chance!

#### **RoboWars**
Prepare for glorious robotic combat! It’s like WWE—only with robots and less spandex. Witness sparks fly as metal crunches against metal! 

---

## The Robo Family

Our team is a curious blend of **Electrical Wizards**, **Mechanical Maestros**, **ML & AI Artists**, **Cybersecurity Sentries**, **Web Dev Whiz Kids**, and **App Dev Genies**. 

So, what are you waiting for? Join us at RoboVitics and help create the bots of tomorrow!

---

## Careers at RoboVitics

Why work just anywhere when you can join a community that celebrates innovation and, dare we say, robot warfare?! Our members work on awe-inspiring projects that have brought us accolades from across the globe. If you’re passionate about tech, we have the perfect spot waiting just for you!

---

## Connect With Us 

We love sharing our adventures, successes, and the occasional cat meme—join us on our social media platforms to keep in touch! 

- **[Follow us on Facebook](#)**  
- **[Check us out on Twitter](#)**  
- **[Catch us on Instagram](#)**  
- **[Join our LinkedIn](#)**  

---

**As the good old robot proverb goes, “Why just exist when you can innovate?” Join RoboVitics today!** 🤖✨